In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from sklearn.metrics import mean_squared_error,log_loss
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.tree import DecisionTreeRegressor


/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df_train = pd.read_csv('./data/train.csv')
df_assignment = pd.read_csv('./data/test.csv')
df = pd.concat([df_train,df_assignment])
df.head()

,alpha,flip_y,id,l1_ratio,max_iter,n_classes,n_clusters_per_class,n_features,n_informative,n_jobs,n_samples,penalty,random_state,scale,time
0,0.0001,0.074798,0,0.304083,417,4,3,327,7,-1,1089,none,475,24.242009,0.409987
1,0.0010,0.077781,1,0.727744,578,4,5,373,7,1,790,l1,569,54.626302,3.950953
2,0.0100,0.030196,2,0.745885,588,2,5,1198,6,2,428,none,529,17.999964,0.368702
3,0.0010,0.057261,3,0.474605,829,6,5,313,7,4,877,none,103,82.257222,1.004559
4,0.0010,0.073728,4,0.395049,167,8,5,644,11,2,216,elasticnet,418,95.515601,0.802800


In [26]:
#create several attribute
#drop some useless attributes
df_train = pd.read_csv('./data/train.csv')
df_assignment = pd.read_csv('./data/test.csv')
df = pd.concat([df_train,df_assignment])
df = df.drop('random_state',axis = 1)
df = df.drop('id',axis=1)
#df['n_jobs'] = np.where((df['n_jobs']==-1)&(df['n_classes']==2),1,df['n_jobs'])
df['n_jobs'] = np.where(df['n_jobs']==-1,16,df['n_jobs'])
df['att1'] = df['n_samples']*df['n_features']*df['n_classes']
df['att2'] = df['n_clusters_per_class']*df['n_classes']/df['n_jobs']
#df['att3'] = df['n_samples']*df['n_features']*df['max_iter']
#df['att4'] = (df['n_features']/df['scale'])
df['att5'] = df['n_samples']*df['n_features']*df['n_classes']*df['max_iter']/df['n_jobs']
df = df.drop('alpha',axis=1)
df = df.drop('n_clusters_per_class',axis=1)
df = df.drop('l1_ratio',axis=1)
df = df.drop('n_classes',axis=1)
df = df.drop('max_iter',axis=1)
df = df.drop('flip_y',axis=1)
df = df.drop('n_features',axis=1)
df = df.drop('n_informative',axis=1)
#df = df.drop('n_jobs',axis=1)
#df = df.drop('n_samples',axis=1)
#df = df.drop('penalty',axis=1)
df = df.drop('scale',axis=1)
df_dummies = pd.get_dummies(df)
df_train_dummies = df_dummies.iloc[:df_train.shape[0],:]
y = df_train_dummies['time']
X = df_train_dummies.drop('time',axis=1)
df_assignment_dummies = df_dummies.iloc[df_train.shape[0]:,:]
X_assignment = df_assignment_dummies.drop('time',axis=1)

In [27]:
#train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state =5001)

In [28]:
#standard scaler to scale the input
scaler = StandardScaler()
df_train_dummies_scale = scaler.fit_transform(X_train)
df_test_dummies_scale = scaler.transform(X_test)
df_assignment_dummies_scale = scaler.transform(X_assignment)

In [30]:
#use gridsearchCV to seach hyperparameters
#use GradientBoostingRegressor for prediction
#y (which is the time) is log 
#the log result will become both negative and positive
#major errors occure when y is large, so sample_weight is used to put more weight in large y

parameters = { 'n_estimators':[250,500,1000],'learning_rate':[0.01,0.5,0.1],'max_depth':[1,2]}
#parameters = { 'n_estimators':[250,500,1000],'learning_rate':[0.01,0.1]}

#est = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1,max_depth=1, random_state=0, loss='ls').fit(df_train_dummies_scale, np.log(y_train))
est = GradientBoostingRegressor( random_state=0, loss='ls')
#est = AdaBoostRegressor(DecisionTreeRegressor(max_depth=2),n_estimators=300, random_state=0)
clf = GridSearchCV(est, parameters, cv=3)
clf.fit(df_train_dummies_scale, np.log(y_train),sample_weight=y_train**3)
#est = AdaBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=0, loss='linear')
#print(cross_val_score(est,df_train_dummies_scale, np.log(y_train),cv=3))
#est.fit(df_train_dummies_scale, np.log(y_train),sample_weight = y_train**2)
#print(mean_squared_error((y_test), np.exp(est.predict(df_test_dummies_scale))))



GridSearchCV(cv=3, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [250, 500, 1000], 'max_depth': [1, 2], 'learning_rate': [0.01, 0.5, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [31]:
#get the hyperparameters associated with the best estimator from gridsearchCV
clf.best_params_

{'learning_rate': 0.5, 'max_depth': 1, 'n_estimators': 1000}

In [34]:
#train the train set and test with the test set with those hyperparamters

best_est = GradientBoostingRegressor(n_estimators = 1000,learning_rate = 0.5,max_depth=1, random_state=0, loss='ls')
#best_est =AdaBoostRegressor(LinearRegression(),n_estimators=500, random_state=0,learning_rate=0.01)
print(cross_val_score(best_est,df_train_dummies_scale, np.log(y_train),cv=3,scoring = 'neg_mean_squared_error'))
best_est.fit(df_train_dummies_scale, np.log(y_train),sample_weight = y_train**3)
print(mean_squared_error((y_train), np.exp(best_est.predict(df_train_dummies_scale))))
print(mean_squared_error((y_test), np.exp(best_est.predict(df_test_dummies_scale))))

#[-0.09014734 -0.05629051 -0.07539125]
#0.15822234011556038
#0.5716804355655117


[-0.09014734 -0.05629051 -0.07539125]
0.15822234011556038
0.5716804355655117


In [39]:
#train again with whole set of data 

best_est = GradientBoostingRegressor(n_estimators = 1000,learning_rate = 0.5,max_depth=1, loss='ls')
best_est.fit(scaler.transform(X), np.log(y),sample_weight = y**3)
df_submission = pd.read_csv('./data/samplesubmission.csv')
df_submission['time'] = np.exp(best_est.predict(df_assignment_dummies_scale))
df_submission.to_csv('12thsubmission.csv',index=False)